# 懒人听书

In [1]:
# https://vb.wting.info/jiankang/jiankangketang/ylsm/lrrvph0lmqt.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1616744423122&t=2&res=640131066&source=web&sign=72d9401e7570be2dddc7cf00dbdddb7e

In [27]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver

In [28]:
def cookie_to_dict(cookie):
    cookie_dict = {}
    items = cookie.split(';')
    for item in items:
        key = item.split('=')[0].replace(' ', '')
        value = item.split('=')[1]
        cookie_dict[key] = value
    return cookie_dict

In [29]:
Cookie = cookie_to_dict('Hm_lvt_ada61571fd48bb3f905f5fd1d6ef0ec4=1616740761; uid=161698172174171f1f9bd6ace40978255d63c2af73ff6; CNZZDATA1254668430=1340647206-1616735582-%7C1616985673; acw_tc=2760779b16169875231224563ea3d362282817c6ca8319abd86cf757959403; Hm_lpvt_ada61571fd48bb3f905f5fd1d6ef0ec4=1616988164')

In [30]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36'
    }
    #Cookie = cookie_to_dict('Hm_lvt_ada61571fd48bb3f905f5fd1d6ef0ec4=1616740761; uid=161698172174171f1f9bd6ace40978255d63c2af73ff6; CNZZDATA1254668430=1340647206-1616735582-%7C1616985673; acw_tc=2760779b16169875231224563ea3d362282817c6ca8319abd86cf757959403; Hm_lpvt_ada61571fd48bb3f905f5fd1d6ef0ec4=1616988164')
    #response = requests.get(url=url, headers=headers, cookies=Cookie)
    response = requests.get(url=url, headers=headers)
    return response

In [31]:
# 用于获取视频源地址的Response
# 懒人听书 使用了referer反爬虫机制 防盗链
def getResponseWithReferer(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36',
        'Referer':'http://www.lrts.me/playlist'
    }
    response = requests.get(url=url, headers=headers)
    return response

In [32]:
# 通过音频源地址爬取音频
def getAudio(url):
    response = getResponse(url)
    path = 'audio/'
    fileName = url.split('?')[0]
    fileName = fileName.split('/')[-1]
    with open(path + fileName, 'wb') as fp:
        fp.write(response.content)
    print(fileName, "Successful")

In [33]:
# 主页面
# http://www.lrts.me/book/category/3086/recommend/1/20
# http://www.lrts.me/book/category/3086/recommend/19/20

In [34]:
# 获取一个页面所有的书的地址
# href="/book/5268"
def getAllLinkOfEveryPage(url):
    html = getResponse(url).text
    soup = BeautifulSoup(html, 'lxml')
    linkLists = []
    for a in soup.find_all('a', href=True): 
        if a.get_text(strip=True): 
            matchObj = re.match(r'/book/[1-9]', a['href'], re.I)
            if matchObj:
                #print(a['href'])
                linkLists.append('http://www.lrts.me'+a['href'])
    return linkLists

In [8]:
for link in getAllLinkOfEveryPage('http://www.lrts.me/book/category/3086/recommend/6/20'):
    print(link)

http://www.lrts.me/book/34706766
http://www.lrts.me/book/21961807
http://www.lrts.me/book/7759
http://www.lrts.me/book/95321679
http://www.lrts.me/book/43855
http://www.lrts.me/book/99623505
http://www.lrts.me/book/30169425
http://www.lrts.me/book/43857
http://www.lrts.me/book/35522898
http://www.lrts.me/book/43860
http://www.lrts.me/book/64157524
http://www.lrts.me/book/37300565
http://www.lrts.me/book/38436949
http://www.lrts.me/book/43861
http://www.lrts.me/book/10399061
http://www.lrts.me/book/30039
http://www.lrts.me/book/11073623
http://www.lrts.me/book/57901401
http://www.lrts.me/book/38745
http://www.lrts.me/book/38493274


In [35]:
def getAllId(url):
    html = getResponse(url).text
    ex = 'id="section(.*?)">'
    all_id = re.findall(ex, html, re.S)
    return all_id

In [36]:
# 获取音频源地址
# http://www.lrts.me/ajax/path/4/82698599/217521422

In [37]:
def getAudioSrcUrl(url):
    # http://www.lrts.me/ajax/path/4/28419739/257411451
    obj_json = getResponseWithReferer(url=url).json()
    return obj_json['data']

In [20]:
# http://www.lrts.me/ajax/playlist/2/28419739/10/next
all_id = getAllId('http://www.lrts.me/ajax/playlist/2/28419739/10/next')
print(all_id)

['257741749', '258821391', '258871223', '258921684', '258971401', '259021013', '259081655', '259131777', '259191194', '259251189']


In [21]:
print(getAudioSrcUrl('http://www.lrts.me/ajax/path/4/28419739/257411451'))

https://vb.wting.info/jiankang/jiankangketang/czbjbldgbjkzbs/lry9nk2skzy.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617071955688&t=2&res=257411451&source=web&sign=5c239f175a3db2a67cda9579da9cf679


In [38]:
# 获取所有书的播放地址
# 1 -- 》 19

pageList = []

with open('bookUrl.txt', 'w', encoding='utf-8') as fp:
    for i in range(1, 20):
        url = f'http://www.lrts.me/book/category/3086/recommend/{i}/20'
        bookList = getAllLinkOfEveryPage(url=url)
        pageList.append(bookList)
        allLink = '\n'.join(bookList)
        fp.write(allLink)
        fp.write('\n')
        print(f'page {i}')
        for link in bookList:
            print(link)

page 1
http://www.lrts.me/book/5268
http://www.lrts.me/book/5269
http://www.lrts.me/book/5271
http://www.lrts.me/book/5272
http://www.lrts.me/book/5273
http://www.lrts.me/book/5274
http://www.lrts.me/book/5275
http://www.lrts.me/book/5276
http://www.lrts.me/book/5277
http://www.lrts.me/book/5278
http://www.lrts.me/book/5279
http://www.lrts.me/book/5280
http://www.lrts.me/book/33280
http://www.lrts.me/book/16852481
http://www.lrts.me/book/91324417
http://www.lrts.me/book/33281
http://www.lrts.me/book/90813697
http://www.lrts.me/book/33282
http://www.lrts.me/book/50013188
http://www.lrts.me/book/50328324
page 2
http://www.lrts.me/book/11524869
http://www.lrts.me/book/3591
http://www.lrts.me/book/3592
http://www.lrts.me/book/16883208
http://www.lrts.me/book/3593
http://www.lrts.me/book/80256521
http://www.lrts.me/book/31037962
http://www.lrts.me/book/44885514
http://www.lrts.me/book/16730891
http://www.lrts.me/book/35647755
http://www.lrts.me/book/14160908
http://www.lrts.me/book/48860684

page 14
http://www.lrts.me/book/41370
http://www.lrts.me/book/41371
http://www.lrts.me/book/28419739
http://www.lrts.me/book/41372
http://www.lrts.me/book/41373
http://www.lrts.me/book/95573663
http://www.lrts.me/book/33686944
http://www.lrts.me/book/62827681
http://www.lrts.me/book/13572258
http://www.lrts.me/book/54352547
http://www.lrts.me/book/47267
http://www.lrts.me/book/26157476
http://www.lrts.me/book/73871524
http://www.lrts.me/book/65479077
http://www.lrts.me/book/55195302
http://www.lrts.me/book/74798502
http://www.lrts.me/book/23485606
http://www.lrts.me/book/47669927
http://www.lrts.me/book/10658473
http://www.lrts.me/book/23513003
page 15
http://www.lrts.me/book/90833325
http://www.lrts.me/book/76228014
http://www.lrts.me/book/89836207
http://www.lrts.me/book/39936688
http://www.lrts.me/book/24317361
http://www.lrts.me/book/34739
http://www.lrts.me/book/51285427
http://www.lrts.me/book/34740
http://www.lrts.me/book/42676
http://www.lrts.me/book/95232949
http://www.lrts.me

In [25]:
url = 'http://www.lrts.me/book/33279'
bookId = url.split('/')[-1]
# http://www.lrts.me/ajax/playlist/2/33279/13/next
# http://www.lrts.me/ajax/path/4/33279/581731175
cnt = 1
while True:
    nextUrl = f'http://www.lrts.me/ajax/playlist/2/{bookId}/{cnt}/next'
    all_id = getAllId(nextUrl)
    if len(all_id) == 0:
        break
    for section in all_id:
        forAudioUrl = f'http://www.lrts.me/ajax/path/4/{bookId}/{section}'
        srcUrl = getAudioSrcUrl(forAudioUrl)
        print(srcUrl)
    cnt = cnt + 10

https://vb.wting.info/asdb/jiankang/qyjyszd/s4nv11bz.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073369761&t=2&res=581061178&source=web&sign=f17de454fcda1aa8faf337e13614f550
https://vb.wting.info/asdb/jiankang/qyjyszd/adu079m0.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073369838&t=2&res=581481115&source=web&sign=bbb6ab75f948976381f5b6391c9b6f34
https://vb.wting.info/asdb/jiankang/qyjyszd/7jekin6v.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073369952&t=2&res=581501964&source=web&sign=6b1a1cc7fa992c840b4c321dcd5039cc
https://vb.wting.info/asdb/jiankang/qyjyszd/3leq1e64.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073370032&t=2&res=581531346&source=web&sign=28a02595ba5fd1378744dc5dff3d61ac
https://vb.wting.info/asdb/jiankang/qyjyszd/a45g4lib.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073370118&t=2&res=581551541&source=web&sign=0a60c9eda921fa9546

In [39]:
print(len(pageList))

19


In [40]:
for i, bookList in enumerate(pageList):
    for j, link in enumerate(bookList):
        print(f'page {i} book {j}')
        bookId = link.split('/')[-1]
        cnt = 1
        while True:
            nextUrl = f'http://www.lrts.me/ajax/playlist/2/{bookId}/{cnt}/next'
            all_id = getAllId(nextUrl)
            if len(all_id) == 0:
                break
            for section in all_id:
                forAudioUrl = f'http://www.lrts.me/ajax/path/4/{bookId}/{section}'
                srcUrl = getAudioSrcUrl(forAudioUrl)
                print(srcUrl)
                getAudio(srcUrl)
            cnt = cnt + 10

page 0 book 0
https://vb.wting.info/asdb/other/snbcj/0tfu27kz.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073893471&t=2&res=637563&source=web&sign=60712a539056dbf0aa7c87b0fe94f14f
0tfu27kz.mp3 Successful
https://vb.wting.info/asdb/other/snbcj/6f506jd1.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073894887&t=2&res=637564&source=web&sign=f3d781fda8ef66a14d1075d3b51e18b7
6f506jd1.mp3 Successful
https://vb.wting.info/asdb/other/snbcj/iv3kwpr6.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073897291&t=2&res=637565&source=web&sign=9098177af02274db2cc6499a062b55a7
iv3kwpr6.mp3 Successful
https://vb.wting.info/asdb/other/snbcj/p810gpct.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073900731&t=2&res=637566&source=web&sign=6d203f3d5e024b99bf16a6a5aa95cb1c
p810gpct.mp3 Successful
https://vb.wting.info/asdb/other/snbcj/urnrusxn.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4V

4btufnrt.mp3 Successful
https://vb.wting.info/asdb/other/yizongjj/5plmkieq.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617073999990&t=2&res=637538&source=web&sign=29f16583c7c8ea8f940e8d0512cf1f65
5plmkieq.mp3 Successful
https://vb.wting.info/asdb/other/yizongjj/uugek7yd.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074004345&t=2&res=637539&source=web&sign=633aff3c1af8034153d374ee8fb2c5b1
uugek7yd.mp3 Successful
https://vb.wting.info/asdb/other/yizongjj/h5h6qkzb.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074006862&t=2&res=637540&source=web&sign=c1744f6d12ea720134cde0ae539b0b69
h5h6qkzb.mp3 Successful
https://vb.wting.info/asdb/other/yizongjj/d29oeq26.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074011211&t=2&res=637541&source=web&sign=824ea3c1768ee76866a67473aa9bb523
d29oeq26.mp3 Successful
https://vb.wting.info/asdb/other/yizongjj/l2dgstl7.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8

bnbl1rc7.mp3 Successful
https://vb.wting.info/asdb/english/spoken/yykyw/3kerk8n3.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074090986&t=2&res=637321&source=web&sign=2c02b84a479a82d92505c1f53c93b941
3kerk8n3.mp3 Successful
https://vb.wting.info/asdb/child/ysyyla/bnbl1rc7.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074091344&t=2&res=637322&source=web&sign=b88607c44c4d1475fc17bd6c50d1d198
bnbl1rc7.mp3 Successful
https://vb.wting.info/asdb/english/spoken/yykyw/3kerk8n3.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074091744&t=2&res=637323&source=web&sign=d80c70fa7b258c37bff717562d869261
3kerk8n3.mp3 Successful
https://vb.wting.info/asdb/english/spoken/yykyw/3kerk8n3.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074092048&t=2&res=637324&source=web&sign=d15185ce02b11cbbe56aa82dcdfeef33
3kerk8n3.mp3 Successful
https://vb.wting.info/asdb/fiction/lishijunshi/lzdt311-330/15opdc2w.mp3?

3kerk8n3.mp3 Successful
https://vb.wting.info/asdb/fiction/chuanyue/mingchaodangwangye/d0nodq0x.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074148819&t=2&res=637358&source=web&sign=235da289d9b18708e92e5eaf9b52c0a9
d0nodq0x.mp3 Successful
https://vb.wting.info/asdb/child/ysyyla/bnbl1rc7.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074149194&t=2&res=637359&source=web&sign=8b1a33e5f9710d4ef00999597163a21b
bnbl1rc7.mp3 Successful
https://vb.wting.info/asdb/child/ysyyla/bnbl1rc7.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074149436&t=2&res=637360&source=web&sign=d2fdc1eaad6270bf935deae1d1f29b3e
bnbl1rc7.mp3 Successful
https://vb.wting.info/asdb/other/redianzh/5162X-724198976.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074149734&t=2&res=637361&source=web&sign=c4cb92fbdf566f2d11ca5c33c1e867a6
5162X-724198976.mp3 Successful
https://vb.wting.info/asdb/child/cndwlxj/sg07poap.mp3?tok

i4pmm57j.mp3 Successful
https://vb.wting.info/asdb/other/sishenxy/2fxdbrl6.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074211345&t=2&res=637395&source=web&sign=b73173a7ae30c34a4475f962352a6f94
2fxdbrl6.mp3 Successful
https://vb.wting.info/asdb/other/sishenxy/iu6ox5ne.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074212767&t=2&res=637396&source=web&sign=313a7b3e81836420e64690301dfbc028
iu6ox5ne.mp3 Successful
https://vb.wting.info/asdb/other/sishenxy/2xq4mwjc.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074215292&t=2&res=637397&source=web&sign=a7a4ecfe633d9d4b126295c92b3b3b6e
2xq4mwjc.mp3 Successful
https://vb.wting.info/asdb/other/sishenxy/xix3bt8p.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617074216731&t=2&res=637398&source=web&sign=298b1d3cff53c805b4c5d4d592b4de75
xix3bt8p.mp3 Successful
https://vb.wting.info/asdb/other/sishenxy/08hx9888.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8

...

6se1ih9u.mp3 Successful
https://vb.wting.info/child/qiyanjueju/6se1ih9u.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617077362683&t=2&res=592231353&source=web&sign=334c91904dd56a3b315a3d6838879e92
6se1ih9u.mp3 Successful
https://vb.wting.info/asdb/fiction/chuanyue/mingchaodangwangye/d0nodq0x.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617077363019&t=2&res=592321809&source=web&sign=fe925e95ba318c70632745066947a05d
d0nodq0x.mp3 Successful
https://vb.wting.info/asdb/other/redianzh/5162X-724198976.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617077363330&t=2&res=592391589&source=web&sign=333f17ef08272f8f616b44e202080c8e
5162X-724198976.mp3 Successful
https://vb.wting.info/asdb/other/shehuibt/5178X1261767425.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617077363570&t=2&res=592461187&source=web&sign=2600679a4b0e54e9e550d946821daaaa
5178X1261767425.mp3 Successful
https://vb.wting.info/asdb/eng

5162X-724198976.mp3 Successful
https://vb.wting.info/asdb/fiction/lishijunshi/lzdt311-330/15opdc2w.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617077801846&t=2&res=233771328&source=web&sign=475d115da7faa692daa54c37a0f2355d
15opdc2w.mp3 Successful
https://vb.wting.info/asdb/fiction/chuanyue/mingchaodangwangye/d0nodq0x.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617077823168&t=2&res=233831570&source=web&sign=9ecf334cc3ee639022454764347ec992
d0nodq0x.mp3 Successful
https://vb.wting.info/asdb/child/cndwlxj/sg07poap.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617077844508&t=2&res=233901327&source=web&sign=39b584b74f740559a99bd6bfd4861941
sg07poap.mp3 Successful
https://vb.wting.info/child/qiyanjueju/6se1ih9u.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617077865890&t=2&res=233971273&source=web&sign=8ec984acff14a2044a67567ef90d50a8
6se1ih9u.mp3 Successful
https://vb.wting.info/asdb/fiction/

https://vb.wting.info/asdb/other/shehuibt/5178X1261767425.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617078614729&t=2&res=341831800&source=web&sign=a09920a2738cc6d597a99cbe79444ca9
5178X1261767425.mp3 Successful
https://vb.wting.info/fiction/gudai/dhzsyqf/2rdyv0il.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617078636028&t=2&res=342961845&source=web&sign=0a18c3ed9f626c961c85e6817929b04b
2rdyv0il.mp3 Successful
https://vb.wting.info/asdb/english/spoken/yykyw/3kerk8n3.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617078658038&t=2&res=343051343&source=web&sign=77f2d495836f15367bb763fbf758a8ea
3kerk8n3.mp3 Successful
https://vb.wting.info/fiction/gudai/dhzsyqf/2rdyv0il.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617078679352&t=2&res=343171917&source=web&sign=f12effd85960cccb1a110e960dd986d6
2rdyv0il.mp3 Successful
https://vb.wting.info/fiction/gudai/dhzsyqf/2rdyv0il.mp3?token=rgSRi8GkjMJuj

https://vb.wting.info/asdb/fiction/lishijunshi/lzdt311-330/15opdc2w.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617079471167&t=2&res=861441884&source=web&sign=70849cb3b32db5ab5f1179070f704136
15opdc2w.mp3 Successful
https://vb.wting.info/child/qiyanjueju/6se1ih9u.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617079492505&t=2&res=861531132&source=web&sign=39aee9203cd4912983cdb87579321afa
6se1ih9u.mp3 Successful
https://vb.wting.info/asdb/fiction/chuanyue/mingchaodangwangye/d0nodq0x.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617079513883&t=2&res=861591105&source=web&sign=9255a0e4f3b96fed7507c92a2e6b1068
d0nodq0x.mp3 Successful
https://vb.wting.info/asdb/child/ysyyla/bnbl1rc7.mp3?token=rgSRi8GkjMJuj8Sm5DapaY2QwU8-txmdiXWxaIT3iNxBvrdOhc4Vzw**&e=1617079535310&t=2&res=861651818&source=web&sign=1aeac788b0212f67a84f3ed28ebe1939
bnbl1rc7.mp3 Successful
https://vb.wting.info/asdb/fiction/chuanyue/mingchaodangwangye/d0no

KeyboardInterrupt: 